# Sequence Start Generator

From the matrix solutions found in the previous "CollatzProof-GenerateMatrices" notebook, it was noted by inspection that a simple generator can be created for the first value of each solution for each rank.

So we can find the starting value for each Collatz sequence for which we have a matrix solution directly without having to solve the matrices.


In [2]:
import sys, io
import math
import numpy as np
import pandas as pd
from scipy.optimize import nnls
from fractions import Fraction

pd.set_option('display.max_rows', None)

# The Shortcut form of the Collatz forward functions

In [2]:
def collatzNext(n):
    if n %2 == 0:
        return n//2
    return (3*n + 1)//2

def collatzSeq(n):
    L = [n]
    while n != 1:
        if n %2 == 0:
            m = n//2
            n = m
        else:
            m = (3*n + 1)//2
            n = m
        L.append(n)
    #
    return L
#

def collatzIter(start, iterations):
    n = start
    count = 0
    while count < iterations:
        if n %2 == 0:
            m = n//2
            yield n
            n = m
        else:
            m = (3*n + 1)//2
            yield n
            n = m
        count = count + 1
    #
    return 1


# Sequence Pretty Printer

Show even/odd and location of "1" in a pleasing way

In [3]:
def blocks(sequence):
    for n in sequence:
        if n == 1:
            yield '⚑'
        elif  n %2 == 0:
            yield '□'
        else:
            yield '■'
    

In [8]:
fract = Fraction(27, 3)

In [10]:
fract.numerator

9

# Generating Function

This function generates the starting values of the matrix solutions of the Shortcut Collatz sequences

In [20]:
def sct_to_fraction(sc_tup):
    '''
    Convert a Shortcut Collatz Tuple to a reduced fraction
    '''
    p2, p3, c = sc_tup
    fract = Fraction(2**p2 - c, 3**p3)
    return (fract.numerator, fract.denominator)
#

def filter_int(sc_fraction_list):
    for fract in sc_fraction_list:
        if fract[1] == 1:
            yield fract[0]
#

# tup expresses a rational number of the form:
#  (2**p2 - c) / (3**p3)

def matrix_start_generator(sc_tup):
    '''
    Generate 2 tuples for the next generation from a single tuple from the current generation
    '''
    p2, p3, c = sc_tup
    t1 = (p2 + 1, p3, c)
    t2 = (p2 + 1, p3 + 1, c*3 + 2**p2)
    return t1, t2
#


def next_generation(prior_generation):
    '''
    Create the next generation from the prior.  We order the tuples so that they
    match up with the order of the matrix solutions for convenience.
    '''
    head_list = []
    tail_list = []
    for sc_tup in prior_generation:
        t1, t2 = matrix_start_generator(sc_tup)
        head_list.append(t1)
        tail_list.append(t2)
    return head_list + tail_list
#

In [21]:
def gen_generation(n_generations):
    gen_1 = [(0,0,0)]
    gen_1_fract = list(map(sct_to_fraction, gen_1))
    print(gen_1_fract)
    gen = gen_1
    for i in range(n_generations):
        gen = next_generation(gen)
        gen_fract = list(map(sct_to_fraction, gen))
        print(gen_fract)
#

In [22]:
gen_generation(8)

[(1, 1)]
[(2, 1), (1, 3)]
[(4, 1), (1, 1), (2, 3), (-1, 9)]
[(8, 1), (7, 3), (2, 1), (1, 3), (4, 3), (1, 9), (-2, 9), (-11, 27)]
[(16, 1), (5, 1), (14, 3), (11, 9), (4, 1), (1, 1), (2, 3), (-1, 9), (8, 3), (5, 9), (2, 9), (-7, 27), (-4, 9), (-13, 27), (-22, 27), (-49, 81)]
[(32, 1), (31, 3), (10, 1), (3, 1), (28, 3), (25, 9), (22, 9), (13, 27), (8, 1), (7, 3), (2, 1), (1, 3), (4, 3), (1, 9), (-2, 9), (-11, 27), (16, 3), (13, 9), (10, 9), (1, 27), (4, 9), (-5, 27), (-14, 27), (-41, 81), (-8, 9), (-17, 27), (-26, 27), (-53, 81), (-44, 27), (-71, 81), (-98, 81), (-179, 243)]
[(64, 1), (21, 1), (62, 3), (59, 9), (20, 1), (19, 3), (6, 1), (5, 3), (56, 3), (53, 9), (50, 9), (41, 27), (44, 9), (35, 27), (26, 27), (-1, 81), (16, 1), (5, 1), (14, 3), (11, 9), (4, 1), (1, 1), (2, 3), (-1, 9), (8, 3), (5, 9), (2, 9), (-7, 27), (-4, 9), (-13, 27), (-22, 27), (-49, 81), (32, 3), (29, 9), (26, 9), (17, 27), (20, 9), (11, 27), (2, 27), (-25, 81), (8, 9), (-1, 27), (-10, 27), (-37, 81), (-28, 27), (-5

In [23]:
def gen_generation_filter_ints(n_generations):
    gen_1 = [(0,0,0)]
    gen_1_fract = list(map(sct_to_fraction, gen_1))
    print(list(filter_int(gen_1_fract)))
    gen = gen_1
    for i in range(n_generations):
        gen = next_generation(gen)
        gen_fract = list(map(sct_to_fraction, gen))
        print(list(filter_int(gen_fract)))
#

In [24]:
gen_generation_filter_ints(8)

[1]
[2]
[4, 1]
[8, 2]
[16, 5, 4, 1]
[32, 10, 3, 8, 2]
[64, 21, 20, 6, 16, 5, 4, 1]
[128, 42, 40, 13, 12, 32, 10, 3, 8, 2]
[256, 85, 84, 80, 26, 24, 64, 21, 20, 6, 16, 5, 4, 1]


In [ ]:
# 25 is the MAX before crashin
gen_generation_filter_ints(24)

[1]
[2]
[4, 1]
[8, 2]
[16, 5, 4, 1]
[32, 10, 3, 8, 2]
[64, 21, 20, 6, 16, 5, 4, 1]
[128, 42, 40, 13, 12, 32, 10, 3, 8, 2]
[256, 85, 84, 80, 26, 24, 64, 21, 20, 6, 16, 5, 4, 1]
[512, 170, 168, 160, 53, 52, 17, 48, 128, 42, 40, 13, 12, 32, 10, 3, 8, 2]
[1024, 341, 340, 113, 336, 320, 106, 35, 104, 34, 11, 96, 256, 85, 84, 80, 26, 24, 64, 21, 20, 6, 16, 5, 4, 1]
[2048, 682, 227, 680, 226, 75, 672, 640, 213, 212, 70, 23, 208, 69, 68, 22, 7, 192, 512, 170, 168, 160, 53, 52, 17, 48, 128, 42, 40, 13, 12, 32, 10, 3, 8, 2]
[4096, 1365, 1364, 454, 151, 1360, 453, 452, 150, 1344, 1280, 426, 424, 141, 140, 46, 15, 416, 138, 136, 45, 44, 14, 384, 1024, 341, 340, 113, 336, 320, 106, 35, 104, 34, 11, 96, 256, 85, 84, 80, 26, 24, 64, 21, 20, 6, 16, 5, 4, 1]
[8192, 2730, 2728, 909, 908, 302, 2720, 906, 904, 301, 300, 2688, 2560, 853, 852, 848, 282, 280, 93, 92, 30, 832, 277, 276, 272, 90, 88, 29, 28, 9, 768, 2048, 682, 227, 680, 226, 75, 672, 640, 213, 212, 70, 23, 208, 69, 68, 22, 7, 192, 512, 170, 16

[2097152, 699050, 699048, 699040, 233013, 233012, 77670, 233008, 77669, 77668, 25889, 699008, 233002, 77667, 233000, 77666, 232992, 698880, 698368, 232789, 232788, 232784, 77594, 77592, 232768, 77589, 77588, 25862, 77584, 25861, 25860, 232704, 232448, 77482, 25827, 77480, 25826, 77472, 77440, 25813, 25812, 25808, 8602, 2867, 8600, 2866, 955, 25792, 8597, 8596, 2865, 8592, 77312, 25770, 25768, 8589, 8588, 2862, 25760, 8586, 8584, 2861, 2860, 953, 25728, 696320, 232106, 232104, 232096, 77365, 77364, 77360, 25786, 8595, 25784, 8594, 232064, 77354, 77352, 77344, 25781, 25780, 8593, 25776, 231936, 231424, 77141, 77140, 25713, 77136, 77120, 25706, 25704, 25696, 8565, 8564, 2854, 951, 8560, 2853, 2852, 950, 77056, 25685, 25684, 8561, 25680, 25664, 8554, 2851, 8552, 2850, 8544, 76800, 688128, 655360, 218453, 218452, 72817, 218448, 218432, 72810, 72808, 24269, 24268, 8089, 72800, 24266, 24264, 218368, 72789, 72788, 24262, 8087, 72784, 24261, 24260, 8086, 2695, 72768, 218112, 217088, 72362, 7236

[8388608, 2796202, 932067, 2796200, 932066, 2796192, 2796160, 932053, 932052, 932048, 310682, 310680, 932032, 310677, 310676, 103558, 34519, 310672, 103557, 103556, 34518, 2796032, 932010, 932008, 310669, 310668, 932000, 310666, 103555, 310664, 103554, 931968, 2795520, 2793472, 931157, 931156, 310385, 931152, 931136, 310378, 103459, 310376, 103458, 310368, 931072, 310357, 310356, 310352, 103450, 34483, 103448, 34482, 310336, 103445, 103444, 34481, 103440, 930816, 929792, 309930, 309928, 103309, 103308, 309920, 103306, 34435, 103304, 34434, 309888, 309760, 103253, 103252, 34417, 103248, 103232, 34410, 34408, 11469, 11468, 3822, 34400, 11466, 11464, 3821, 3820, 1273, 103168, 34389, 34388, 11462, 34384, 11461, 11460, 34368, 309248, 103082, 103080, 103072, 34357, 34356, 34352, 11450, 11448, 103040, 34346, 34344, 34336, 11445, 11444, 3814, 1271, 11440, 3813, 3812, 1270, 423, 102912, 2785280, 928426, 309475, 928424, 309474, 928416, 928384, 309461, 309460, 103153, 309456, 309440, 103146, 1031

[33554432, 11184810, 11184808, 3728269, 3728268, 11184800, 3728266, 1242755, 3728264, 1242754, 414251, 11184768, 11184640, 3728213, 3728212, 1242737, 3728208, 3728192, 1242730, 414243, 1242728, 414242, 1242720, 3728128, 1242709, 1242708, 1242704, 414234, 414232, 138077, 138076, 46025, 1242688, 414229, 414228, 414224, 138074, 138072, 11184128, 3728042, 3728040, 3728032, 1242677, 1242676, 414225, 1242672, 3728000, 1242666, 1242664, 414221, 414220, 138073, 1242656, 414218, 414216, 3727872, 11182080, 11173888, 3724629, 3724628, 1241542, 413847, 3724624, 1241541, 1241540, 413846, 3724608, 3724544, 1241514, 1241512, 413837, 413836, 137945, 1241504, 413834, 413832, 1241472, 3724288, 1241429, 1241428, 413809, 1241424, 1241408, 413802, 413800, 137933, 137932, 45977, 413792, 137930, 137928, 1241344, 413781, 413780, 137926, 45975, 413776, 137925, 137924, 45974, 413760, 3723264, 3719168, 1239722, 1239720, 1239712, 413237, 413236, 137745, 413232, 1239680, 413226, 413224, 137741, 137740, 45913, 4132

# Lets try to get more efficient with NumPy Arrays

In [4]:
np.zeros((1,3), dtype=int)

array([[0, 0, 0]])

In [6]:
g = np.zeros((4,3), dtype=int)
g[0] = np.array([1,2,3], dtype=int)
g

array([[1, 2, 3],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [7]:
g.shape

(4, 3)

In [32]:
def generation_fractions(generation):
    fracts = np.zeros((generation.shape[0], 2), dtype=int)
    for i in range(generation.shape[0]):
        p2, p3, c = generation[i]
        fract = Fraction(2**p2 - c, 3**p3)
        fracts[i][0], fracts[i][1] = fract.numerator, fract.denominator
    return fracts
#

def generation_fractions(generation):
    fracts = np.zeros((generation.shape[0], 2), dtype=int)
    for i in range(generation.shape[0]):
        p2, p3, c = generation[i]
        fract = Fraction(2**p2 - c, 3**p3)
        fracts[i][0], fracts[i][1] = fract.numerator, fract.denominator
    return fracts
#

def filter_int_with_tup(generation, fractions):
    for i in range(len(fractions)):
        if fractions[i][1] == 1:
            yield (fractions[i][0], generation[i])
#

def filter_int_with_fraction(sc_fraction_list):
    for fract in sc_fraction_list:
        if fract[1] == 1:
            yield fract[0]
#


# tup expresses a rational number of the form:
#  (2**p2 - c) / (3**p3)

def matrix_start_generator(sc_array):
    '''
    Generate 2 tuples for the next generation from a single tuple from the current generation
    '''
    p2, p3, c = sc_array
    t1 = np.array([p2 + 1, p3, c], dtype=int)
    t2 = np.array([p2 + 1, p3 + 1, c*3 + 2**p2], dtype=int)
    return t1, t2
#


def next_generation(prior_generation):
    '''
    Create the next generation from the prior.  We order the tuples so that they
    match up with the order of the matrix solutions for convenience.
    '''
    prior_shape = prior_generation.shape
    next_shape = (prior_shape[0]*2, 3)
    next_generation = np.zeros(next_shape, dtype=int)
    
    for i in range (prior_shape[0]):
        t1, t2 = matrix_start_generator(prior_generation[i])
        next_generation[i] = t1
        next_generation[i+prior_shape[0]] = t2
    return next_generation
#

In [29]:
def gen_generation(n_generations):
    gen_1 = np.zeros((1,3), dtype=int)
    
    gen_1_fract = generation_fractions(gen_1)
    print(list(filter_int(gen_1_fract)))
    gen = gen_1
    for i in range(n_generations):
        gen = next_generation(gen)
        gen_fract = generation_fractions(gen)
        print(list(filter_int(gen_fract)))
#

In [24]:
gen_generation(24)

[1]
[2]
[4, 1]
[8, 2]
[16, 5, 4, 1]
[32, 10, 3, 8, 2]
[64, 21, 20, 6, 16, 5, 4, 1]
[128, 42, 40, 13, 12, 32, 10, 3, 8, 2]
[256, 85, 84, 80, 26, 24, 64, 21, 20, 6, 16, 5, 4, 1]
[512, 170, 168, 160, 53, 52, 17, 48, 128, 42, 40, 13, 12, 32, 10, 3, 8, 2]
[1024, 341, 340, 113, 336, 320, 106, 35, 104, 34, 11, 96, 256, 85, 84, 80, 26, 24, 64, 21, 20, 6, 16, 5, 4, 1]
[2048, 682, 227, 680, 226, 75, 672, 640, 213, 212, 70, 23, 208, 69, 68, 22, 7, 192, 512, 170, 168, 160, 53, 52, 17, 48, 128, 42, 40, 13, 12, 32, 10, 3, 8, 2]
[4096, 1365, 1364, 454, 151, 1360, 453, 452, 150, 1344, 1280, 426, 424, 141, 140, 46, 15, 416, 138, 136, 45, 44, 14, 384, 1024, 341, 340, 113, 336, 320, 106, 35, 104, 34, 11, 96, 256, 85, 84, 80, 26, 24, 64, 21, 20, 6, 16, 5, 4, 1]
[8192, 2730, 2728, 909, 908, 302, 2720, 906, 904, 301, 300, 2688, 2560, 853, 852, 848, 282, 280, 93, 92, 30, 832, 277, 276, 272, 90, 88, 29, 28, 9, 768, 2048, 682, 227, 680, 226, 75, 672, 640, 213, 212, 70, 23, 208, 69, 68, 22, 7, 192, 512, 170, 16

[2097152, 699050, 699048, 699040, 233013, 233012, 77670, 233008, 77669, 77668, 25889, 699008, 233002, 77667, 233000, 77666, 232992, 698880, 698368, 232789, 232788, 232784, 77594, 77592, 232768, 77589, 77588, 25862, 77584, 25861, 25860, 232704, 232448, 77482, 25827, 77480, 25826, 77472, 77440, 25813, 25812, 25808, 8602, 2867, 8600, 2866, 955, 25792, 8597, 8596, 2865, 8592, 77312, 25770, 25768, 8589, 8588, 2862, 25760, 8586, 8584, 2861, 2860, 953, 25728, 696320, 232106, 232104, 232096, 77365, 77364, 77360, 25786, 8595, 25784, 8594, 232064, 77354, 77352, 77344, 25781, 25780, 8593, 25776, 231936, 231424, 77141, 77140, 25713, 77136, 77120, 25706, 25704, 25696, 8565, 8564, 2854, 951, 8560, 2853, 2852, 950, 77056, 25685, 25684, 8561, 25680, 25664, 8554, 2851, 8552, 2850, 8544, 76800, 688128, 655360, 218453, 218452, 72817, 218448, 218432, 72810, 72808, 24269, 24268, 8089, 72800, 24266, 24264, 218368, 72789, 72788, 24262, 8087, 72784, 24261, 24260, 8086, 2695, 72768, 218112, 217088, 72362, 7236

[8388608, 2796202, 932067, 2796200, 932066, 2796192, 2796160, 932053, 932052, 932048, 310682, 310680, 932032, 310677, 310676, 103558, 34519, 310672, 103557, 103556, 34518, 2796032, 932010, 932008, 310669, 310668, 932000, 310666, 103555, 310664, 103554, 931968, 2795520, 2793472, 931157, 931156, 310385, 931152, 931136, 310378, 103459, 310376, 103458, 310368, 931072, 310357, 310356, 310352, 103450, 34483, 103448, 34482, 310336, 103445, 103444, 34481, 103440, 930816, 929792, 309930, 309928, 103309, 103308, 309920, 103306, 34435, 103304, 34434, 309888, 309760, 103253, 103252, 34417, 103248, 103232, 34410, 34408, 11469, 11468, 3822, 34400, 11466, 11464, 3821, 3820, 1273, 103168, 34389, 34388, 11462, 34384, 11461, 11460, 34368, 309248, 103082, 103080, 103072, 34357, 34356, 34352, 11450, 11448, 103040, 34346, 34344, 34336, 11445, 11444, 3814, 1271, 11440, 3813, 3812, 1270, 423, 102912, 2785280, 928426, 309475, 928424, 309474, 928416, 928384, 309461, 309460, 103153, 309456, 309440, 103146, 1031

In [25]:
def count_generation_ints(n_generations):
    gen_n = 1
    gen_1 = np.zeros((1,3), dtype=int)
    
    gen_1_fract = generation_fractions(gen_1)
    gen_ints = list(filter_int(gen_1_fract))
    print([gen_n, len(gen_ints), gen_ints[0], len(gen_ints)/gen_ints[0]])
    gen = gen_1
    for i in range(n_generations):
        gen = next_generation(gen)
        gen_fract = generation_fractions(gen)
        gen_ints = list(filter_int(gen_fract))
        gen_n = gen_n + 1
        print([gen_n, len(gen_ints), gen_ints[0], len(gen_ints)/gen_ints[0]])
#

In [26]:
count_generation_ints(24)

[1, 1, 1, 1.0]
[2, 1, 2, 0.5]
[3, 2, 4, 0.5]
[4, 2, 8, 0.25]
[5, 4, 16, 0.25]
[6, 5, 32, 0.15625]
[7, 8, 64, 0.125]
[8, 10, 128, 0.078125]
[9, 14, 256, 0.0546875]
[10, 18, 512, 0.03515625]
[11, 26, 1024, 0.025390625]
[12, 36, 2048, 0.017578125]
[13, 50, 4096, 0.01220703125]
[14, 67, 8192, 0.0081787109375]
[15, 89, 16384, 0.00543212890625]
[16, 117, 32768, 0.003570556640625]
[17, 157, 65536, 0.0023956298828125]
[18, 208, 131072, 0.0015869140625]
[19, 277, 262144, 0.001056671142578125]
[20, 367, 524288, 0.0006999969482421875]
[21, 488, 1048576, 0.00046539306640625]
[22, 649, 2097152, 0.0003094673156738281]
[23, 869, 4194304, 0.0002071857452392578]
[24, 1154, 8388608, 0.00013756752014160156]
[25, 1534, 16777216, 9.143352508544922e-05]


In [35]:
def gen_generation_with_tup(n_generations):
    gen_1 = np.zeros((1,3), dtype=int)
    
    gen_1_fract = generation_fractions(gen_1)
    print(list(filter_int_with_tup(gen_1, gen_1_fract)))
    gen = gen_1
    for i in range(n_generations):
        gen = next_generation(gen)
        gen_fract = generation_fractions(gen)
        print(list(filter_int_with_tup(gen, gen_fract)))
#

In [36]:
gen_generation_with_tup(15)

[(1, array([0, 0, 0]))]
[(2, array([1, 0, 0]))]
[(4, array([2, 0, 0])), (1, array([2, 1, 1]))]
[(8, array([3, 0, 0])), (2, array([3, 1, 2]))]
[(16, array([4, 0, 0])), (5, array([4, 1, 1])), (4, array([4, 1, 4])), (1, array([4, 2, 7]))]
[(32, array([5, 0, 0])), (10, array([5, 1, 2])), (3, array([5, 2, 5])), (8, array([5, 1, 8])), (2, array([ 5,  2, 14]))]
[(64, array([6, 0, 0])), (21, array([6, 1, 1])), (20, array([6, 1, 4])), (6, array([ 6,  2, 10])), (16, array([ 6,  1, 16])), (5, array([ 6,  2, 19])), (4, array([ 6,  2, 28])), (1, array([ 6,  3, 37]))]
[(128, array([7, 0, 0])), (42, array([7, 1, 2])), (40, array([7, 1, 8])), (13, array([ 7,  2, 11])), (12, array([ 7,  2, 20])), (32, array([ 7,  1, 32])), (10, array([ 7,  2, 38])), (3, array([ 7,  3, 47])), (8, array([ 7,  2, 56])), (2, array([ 7,  3, 74]))]
[(256, array([8, 0, 0])), (85, array([8, 1, 1])), (84, array([8, 1, 4])), (80, array([ 8,  1, 16])), (26, array([ 8,  2, 22])), (24, array([ 8,  2, 40])), (64, array([ 8,  1, 64])

[(32768, array([15,  0,  0])), (10922, array([15,  1,  2])), (10920, array([15,  1,  8])), (10912, array([15,  1, 32])), (3637, array([15,  2, 35])), (3636, array([15,  2, 44])), (3632, array([15,  2, 80])), (1210, array([15,  3, 98])), (403, array([ 15,   4, 125])), (1208, array([ 15,   3, 152])), (402, array([ 15,   4, 206])), (10880, array([ 15,   1, 128])), (3626, array([ 15,   2, 134])), (3624, array([ 15,   2, 152])), (3616, array([ 15,   2, 224])), (1205, array([ 15,   3, 233])), (1204, array([ 15,   3, 260])), (401, array([ 15,   4, 287])), (1200, array([ 15,   3, 368])), (10752, array([ 15,   1, 512])), (10240, array([  15,    1, 2048])), (3413, array([  15,    2, 2051])), (3412, array([  15,    2, 2060])), (1137, array([  15,    3, 2069])), (3408, array([  15,    2, 2096])), (3392, array([  15,    2, 2240])), (1130, array([  15,    3, 2258])), (1128, array([  15,    3, 2312])), (1120, array([  15,    3, 2528])), (373, array([  15,    4, 2555])), (372, array([  15,    4, 2636]

# Lets Brute Force some integers

Lets iterate over some powers of 2 and see what integers we get

In [72]:
def brute(N):
    for p2 in range(N):
        max_p3 = math.floor(p2*(2.0/3.0))
        for p3 in range(1, max_p3):
            for c in range(2**p2 - 1):
                f = Fraction(2**p2 - c, 3**p3)
                if f.denominator == 1:
                    print(f.numerator, (p2, p3, c))
#

In [73]:
brute(8)

2 (3, 1, 2)
1 (3, 1, 5)
5 (4, 1, 1)
4 (4, 1, 4)
3 (4, 1, 7)
2 (4, 1, 10)
1 (4, 1, 13)
10 (5, 1, 2)
9 (5, 1, 5)
8 (5, 1, 8)
7 (5, 1, 11)
6 (5, 1, 14)
5 (5, 1, 17)
4 (5, 1, 20)
3 (5, 1, 23)
2 (5, 1, 26)
1 (5, 1, 29)
3 (5, 2, 5)
2 (5, 2, 14)
1 (5, 2, 23)
21 (6, 1, 1)
20 (6, 1, 4)
19 (6, 1, 7)
18 (6, 1, 10)
17 (6, 1, 13)
16 (6, 1, 16)
15 (6, 1, 19)
14 (6, 1, 22)
13 (6, 1, 25)
12 (6, 1, 28)
11 (6, 1, 31)
10 (6, 1, 34)
9 (6, 1, 37)
8 (6, 1, 40)
7 (6, 1, 43)
6 (6, 1, 46)
5 (6, 1, 49)
4 (6, 1, 52)
3 (6, 1, 55)
2 (6, 1, 58)
1 (6, 1, 61)
7 (6, 2, 1)
6 (6, 2, 10)
5 (6, 2, 19)
4 (6, 2, 28)
3 (6, 2, 37)
2 (6, 2, 46)
1 (6, 2, 55)
2 (6, 3, 10)
1 (6, 3, 37)
42 (7, 1, 2)
41 (7, 1, 5)
40 (7, 1, 8)
39 (7, 1, 11)
38 (7, 1, 14)
37 (7, 1, 17)
36 (7, 1, 20)
35 (7, 1, 23)
34 (7, 1, 26)
33 (7, 1, 29)
32 (7, 1, 32)
31 (7, 1, 35)
30 (7, 1, 38)
29 (7, 1, 41)
28 (7, 1, 44)
27 (7, 1, 47)
26 (7, 1, 50)
25 (7, 1, 53)
24 (7, 1, 56)
23 (7, 1, 59)
22 (7, 1, 62)
21 (7, 1, 65)
20 (7, 1, 68)
19 (7, 1, 71)
18 (7, 1, 74)
17 